# Introduction

In this notebook I scrape image data from Google, Yandex, and Flickr on Google Chrome. The goal is to add image data to the [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition) data set available on Kaggle with new species of flowers. So far I have used this code to add images of the following flower species to the [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition) data set:

- aster
- orchid

## Load all packages

In [1]:
from PIL import Image
import os
from selenium import webdriver
import time
import io
import requests
import hashlib

## Scraping from Google Chrome

The bit of code below functions to scrape images from Google. This method is based on this post which gives info on how to scrape image data from Google: [https://medium.com/p/a96feda8af2d](https://medium.com/p/a96feda8af2d)

In [2]:
search_term = 'orchid'

In [3]:
DRIVER_PATH = './chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [4]:
wd.get('https://google.com')

In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG")
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def search_and_download(search_term:str,driver_path:str,target_path='./flowers',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In the code below we may have to adjust number of images we can pull from Google. If it finds less than the number of images indicated it will return an error.

In [8]:
img_path = 'flowers/' + search_term + '/'

if search_term not in os.listdir('flowers/.'): 
    os.mkdir(img_path)

search_and_download(search_term=search_term,driver_path=DRIVER_PATH,number_images=160)

Found: 100 search results. Extracting links from 0:100
Found: 161 image links, done!
SUCCESS - saved https://d.newsweek.com/en/full/1888616/stock-image-orchid.jpg - as ./flowers/orchid/dd8ae83ced.jpg
SUCCESS - saved https://s3.amazonaws.com/eit-planttoolbox-prod/media/images/Pink_Phalaenopsis_or_hMoyA5LaLqOb.jpeg - as ./flowers/orchid/b0fb80ace9.jpg
SUCCESS - saved https://gardens.si.edu/wp-content/uploads/2019/05/Cymbidium-pink-1280x720.jpg - as ./flowers/orchid/13d73e7e0e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTheNm_H3_vYn6SuFTaMgxeSYEM0S_Tu0F7gA&usqp=CAU - as ./flowers/orchid/1ae6768943.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQn4ctqtKQetUl-f7a7X5PFU8mwv-vADqwheg&usqp=CAU - as ./flowers/orchid/32c0fbdb3f.jpg
SUCCESS - saved https://cdn.britannica.com/50/5650-050-77351097/Epiphytes-orchids-roots-air-moisture-plants-hosts.jpg - as ./flowers/orchid/3caca387c8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/image

SUCCESS - saved https://www.orchidsbyhausermann.com/mm5/graphics/00000001/shop-in-bud.jpg - as ./flowers/orchid/ca0d0a9d8c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqQ5MOZLSCO58ABFAXAxNUKb7SFwRQXUtLAw&usqp=CAU - as ./flowers/orchid/b334733966.jpg
SUCCESS - saved https://th-thumbnailer.cdn-si-edu.com/VW6LZl2eRoLrDiJT7ws_lvztKVc=/fit-in/1600x0/filters:focal(605x457:606x458)/https://tf-cmsv2-smithsonianmag-media.s3.amazonaws.com/filer/bf/0c/bf0cab36-0f3f-4786-812c-178ba50385fc/1_web_callan-cohen-disa-forficaria-oz0a0487-1.jpg - as ./flowers/orchid/d3b89ae7e3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnZY_M7DgYGiPJMn-bCVc-KUQCSWLHLHilug&usqp=CAU - as ./flowers/orchid/d0a06a314b.jpg
SUCCESS - saved https://www.thespruce.com/thmb/-qDVr0Setf659lwiLGF7qkyFjqk=/1969x1969/smart/filters:no_upscale()/how-to-repot-orchids-1902834-1-c07cc5c3db3c4875be27f98684bbd2a9.jpg - as ./flowers/orchid/c8018f013e.jpg
ERROR - Could not save https:/

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0G448htvh1rR_Kp9xP0NJ8dkjukmoo0YcDw&usqp=CAU - as ./flowers/orchid/8f7bf273f6.jpg
SUCCESS - saved https://www.englishgardens.com/wp-content/uploads/Blog-Banners-58.png - as ./flowers/orchid/64a3ee1306.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxK6lJ7VRIb8iOL95ju832KfxT96gh7wS2fw&usqp=CAU - as ./flowers/orchid/94377bb86d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRvKkQ2dF2rZloZz8bLsUcCP9arNKfsznAyBw&usqp=CAU - as ./flowers/orchid/c697c7b73c.jpg
SUCCESS - saved https://www.ikea.com/us/en/images/products/fejka-artificial-potted-plant-orchid-white__0748880_pe745269_s5.jpg?f=s - as ./flowers/orchid/cf091ce075.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRJGBUyMv7X6S5weIqD70g4SRhNmwDPI4-MhQ&usqp=CAU - as ./flowers/orchid/9f9f755961.jpg
ERROR - Could not save https://www.kellogggarden.com/wp-content/uploads/2021/07/The-Ultimate-Guid

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTWeQn8y19ajVBjwzSRtyOgqmdH9FvPTPwdQ&usqp=CAU - as ./flowers/orchid/2de427d700.jpg
SUCCESS - saved https://media.istockphoto.com/photos/closeup-of-pink-orchid-flower-isolated-on-white-background-picture-id877067668 - as ./flowers/orchid/c44c66b76c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT2xjoSkJGGEfQMbG4SY1nS6ykE0-cFeZVh4Q&usqp=CAU - as ./flowers/orchid/d4d18a234b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT1sK1hPCHQ22jOBtBf5zRHF6yrq7vckhSm7g&usqp=CAU - as ./flowers/orchid/304d6d7ba1.jpg
SUCCESS - saved https://cityfloralgreenhouse.com/wp-content/uploads/2020/12/orchids-4792220_1280.jpg - as ./flowers/orchid/59d373b479.jpg
SUCCESS - saved https://m.media-amazon.com/images/S/assets.wholefoodsmarket.com/PIE/product/5e4054fbe47f84cd5dd5c894_00033383952253-glamor-frontpackageglamor-2020-01-28t16-47-39-pixel-3a-quality-90-1-20-1-user-5984ad42a967f880524de2c4

## Scraping images on Flickr

The two lines of code search Flickr for the number of images specified after the search term. See the following repository to download, setup, and get 'flickrGetUrl.py' and 'get_images.py' to run: [https://github.com/ultralytics/flickr_scraper](https://github.com/ultralytics/flickr_scraper). The versions of that code that are run here are slightly modified for easier use in this repository.

In [9]:
run flickrGetUrl.py orchid 500

Fetching url for image number 1
Fetching url for image number 2
Fetching url for image number 3
Fetching url for image number 4
Fetching url for image number 5
Fetching url for image number 6
Fetching url for image number 7
Fetching url for image number 8
Fetching url for image number 9
Fetching url for image number 10
Fetching url for image number 11
Fetching url for image number 12
Fetching url for image number 13
Fetching url for image number 14
Fetching url for image number 15
Fetching url for image number 16
Fetching url for image number 17
Fetching url for image number 18
Fetching url for image number 19
Fetching url for image number 20
Fetching url for image number 21
Fetching url for image number 22
Fetching url for image number 23
Fetching url for image number 24
Fetching url for image number 25
Fetching url for image number 26
Fetching url for image number 27
Fetching url for image number 28
Fetching url for image number 29
Fetching url for image number 30
Fetching url for im

Fetching url for image number 251
Fetching url for image number 252
Fetching url for image number 253
Fetching url for image number 254
Fetching url for image number 255
Fetching url for image number 256
Fetching url for image number 257
Fetching url for image number 258
Fetching url for image number 259
Fetching url for image number 260
Fetching url for image number 261
Fetching url for image number 262
Fetching url for image number 263
Fetching url for image number 264
Fetching url for image number 265
Fetching url for image number 266
Fetching url for image number 267
Fetching url for image number 268
Fetching url for image number 269
Fetching url for image number 270
Fetching url for image number 271
Fetching url for image number 272
Fetching url for image number 273
Fetching url for image number 274
Fetching url for image number 275
Fetching url for image number 276
Fetching url for image number 277
Fetching url for image number 278
Fetching url for image number 279
Fetching url f

Done fetching urls, fetched 500 urls out of 500
Writing out the urls in the current directory
Done!!!


In [10]:
run get_images.py image_urls.csv orchid

Starting download 1 of  317
Done downloading 1 of 317
Starting download 2 of  317
Done downloading 2 of 317
Starting download 3 of  317
Done downloading 3 of 317
Starting download 4 of  317
Done downloading 4 of 317
Starting download 5 of  317
Done downloading 5 of 317
Starting download 6 of  317
Done downloading 6 of 317
Starting download 7 of  317
Done downloading 7 of 317
Starting download 8 of  317
Done downloading 8 of 317
Starting download 9 of  317
Done downloading 9 of 317
Starting download 10 of  317
Done downloading 10 of 317
Starting download 11 of  317
Done downloading 11 of 317
Starting download 12 of  317
Done downloading 12 of 317
Starting download 13 of  317
Done downloading 13 of 317
Starting download 14 of  317
Done downloading 14 of 317
Starting download 15 of  317
Done downloading 15 of 317
Starting download 16 of  317
Done downloading 16 of 317
Starting download 17 of  317
Done downloading 17 of 317
Starting download 18 of  317
Done downloading 18 of 317
Starting d

Done downloading 146 of 317
Starting download 147 of  317
Done downloading 147 of 317
Starting download 148 of  317
Done downloading 148 of 317
Starting download 149 of  317
Done downloading 149 of 317
Starting download 150 of  317
Done downloading 150 of 317
Starting download 151 of  317
Done downloading 151 of 317
Starting download 152 of  317
Done downloading 152 of 317
Starting download 153 of  317
Done downloading 153 of 317
Starting download 154 of  317
Done downloading 154 of 317
Starting download 155 of  317
Done downloading 155 of 317
Starting download 156 of  317
Done downloading 156 of 317
Starting download 157 of  317
Done downloading 157 of 317
Starting download 158 of  317
Done downloading 158 of 317
Starting download 159 of  317
Done downloading 159 of 317
Starting download 160 of  317
Done downloading 160 of 317
Starting download 161 of  317
Done downloading 161 of 317
Starting download 162 of  317
Done downloading 162 of 317
Starting download 163 of  317
Done downloadi

Done downloading 288 of 317
Starting download 289 of  317
Done downloading 289 of 317
Starting download 290 of  317
Done downloading 290 of 317
Starting download 291 of  317
Done downloading 291 of 317
Starting download 292 of  317
Done downloading 292 of 317
Starting download 293 of  317
Done downloading 293 of 317
Starting download 294 of  317
Done downloading 294 of 317
Starting download 295 of  317
Done downloading 295 of 317
Starting download 296 of  317
Done downloading 296 of 317
Starting download 297 of  317
Done downloading 297 of 317
Starting download 298 of  317
Done downloading 298 of 317
Starting download 299 of  317
Done downloading 299 of 317
Starting download 300 of  317
Done downloading 300 of 317
Starting download 301 of  317
Done downloading 301 of 317
Starting download 302 of  317
Done downloading 302 of 317
Starting download 303 of  317
Done downloading 303 of 317
Starting download 304 of  317
Done downloading 304 of 317
Starting download 305 of  317
Done downloadi

## Scraping images on Yandex

The next line of code searches Yandex for images of the search term and puts them in a temporary folder named 'downloads' which I then delete after moving the images out of there. Note that these are run outside of Python (hence the ! before each command).

In [11]:
! yandex-images-download Chrome --keywords "orchid" --limit 500 

Output directory is set to "downloads/"
Limit of images is set to 500
  Found 50 pages of orchid.
  Scrapping page 1/17...
    Downloaded the image. ==> downloads/orchid/orxideya-makro-ekzotika.jpg
    Downloaded the image. ==> downloads/orchid/Phaedriels-orchid.jpeg
    Downloaded the image. ==> downloads/orchid/u-orhidei-myagkie-i-vyalye-listya-prichiny-i-leche.jpg
    Downloaded the image. ==> downloads/orchid/oshibkivuhodorchid11.jpg
    Downloaded the image. ==> downloads/orchid/1617729890_4-p-oboi-orkhideya-falenopsis-rastsvetk.jpg
    Downloaded the image. ==> downloads/orchid/57d49854a633b451c347acb6c8a17bdb.jpg
    Downloaded the image. ==> downloads/orchid/orhideya-simvoliziruet-semeynyy-uyut-i-blagopoluch.jpg
    Downloaded the image. ==> downloads/orchid/screen-18.jpg
    fail: https://upload.wikimedia.org/wikipedia/commons/3/30/Orchid_Phalaenopsis_hybrid.jpg error: img_url response is not ok. response: <Response [403]>.
    fail: https://catherineasquithgallery.com/uploads

  Scrapping page 4/17...
    Downloaded the image. ==> downloads/orchid/orchid-1059672_1280.jpg
    Downloaded the image. ==> downloads/orchid/original-7.jpg
    Downloaded the image. ==> downloads/orchid/orxidei-ekzotika-makro.jpg
    Downloaded the image. ==> downloads/orchid/thumb_l_3336.jpg
    Downloaded the image. ==> downloads/orchid/1335708720_arhidei-95.jpg
    Downloaded the image. ==> downloads/orchid/e2ea7135e48af0c5dccaadf9ea02001f.jpg
    Downloaded the image. ==> downloads/orchid/1612258373_90-p-orkhidei-na-fioletovom-fone-119.jpg
    Downloaded the image. ==> downloads/orchid/967d994b0f3db1fc35485ceb6b5b3928.jpg
    Downloaded the image. ==> downloads/orchid/1335708694_arhidei-85.jpg
    Downloaded the image. ==> downloads/orchid/upl_1614825026_18874_weqk8.jpg
    fail: https://images.wallpaperscraft.com/image/orchid_flower_thistle_close-up_42888_1280x1024.jpg error: img_url response is not ok. response: <Response [404]>.
    Downloaded the image. ==> downloads/orchid/o

    fail: https://s1.1zoom.ru/b5050/424/Closeup_Orchid_Violet_503771_2560x1600.jpg error: img_url response is not ok. response: <Response [404]>.
    Downloaded the image. ==> downloads/orchid/14977745533319_1920x1200.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/scale_1200 (6).jpg
    Downloaded the image. ==> downloads/orchid/1614434240_72-p-orkhidei-na-temnom-fone-89.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/4K-Orchid-Wallpaper-Free.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/Closeup_Orchid_570756_1920x1200.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/orchid-1841879_1280.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/1617609911_3-p-orkhidei-v-interere-4.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/flower-1236843_1920_result.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/orchids-2059045_1280.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/1612258268_8-p-orkhidei-na-fioletovom-f

    Downloaded the image. ==> downloads/orchid/orchid-715907_1280.jpg
    Downloaded the image. ==> downloads/orchid/orhideja-vanda.jpg
    Downloaded the image. ==> downloads/orchid/ae9bbd5c97966ffe7b5d374c639d76d1.jpg
    Downloaded the image. ==> downloads/orchid/branch-blossom-plant-leaf-flower-petal-bloom-flora.jpg
    Downloaded the image. ==> downloads/orchid/1612258305_17-p-orkhidei-na-fioletovom-fone-22.jpg
    Downloaded the image. ==> downloads/orchid/515ff4bac50ddc88d4356265b6dc78b4.jpg
    Downloaded the image. ==> downloads/orchid/00cc785c1673d7f5705772424449d77f.jpg
    fail: https://orhidey.com/wp-content/gallery/white-orhidey/Orchid-5.JPG error: <class 'requests.exceptions.ConnectionError'>
    Downloaded the image. ==> downloads/orchid/scale_1200 (9).jpg
    Downloaded the image. ==> downloads/orchid/butony-orhidea.jpg
    Downloaded the image. ==> downloads/orchid/1612483193_79-p-orkhidei-na-serom-fone-127.jpg
    fail: https://florino.net/wp-content/uploads/2017/07/

    Downloaded the image. ==> downloads/orchid/s1200-2-12.jpg
    Downloaded the image. ==> downloads/orchid/cvetok-buket-cvety-nezhnye-prekrasnyj-orxidei.jpg
    Downloaded the image. ==> downloads/orchid/orhideya-krasivaya-orhideya.jpg
    Downloaded the image. ==> downloads/orchid/28526.jpg
    Downloaded the image. ==> downloads/orchid/orchid-1707074_1280.jpg
    Downloaded the image. ==> downloads/orchid/30eb58aac1241dddcefb62dd04eb0ca8.jpg
    fail: https://www.fonstola.ru/download.php?file=201305/640x960/fonstola.ru-95326.jpg error: ('Something is wrong here.', " Error: (<class 'KeyError'>, KeyError('application/octet-stream'))")
    Downloaded the image. ==> downloads/orchid/ce16b10ed2a0e260499a840a11516954.jpg
    Downloaded the image. ==> downloads/orchid/2048x1365_1065393_[www.ArtFile.ru].jpg
    fail: https://florino.net/wp-content/uploads/2017/07/orchid-1677518_1920.jpg error: img_url response is not ok. response: <Response [403]>.
    Downloaded the image. ==> downloads/o

    Downloaded the image. ==> downloads/orchid/224429-svetik.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/photo-market-aff0d12f.jpg (1).jpg
    fail: https://www.espoma.com/wp-content/uploads/2017/01/phalaenopsis-621906_1920.jpg error: img_url response is not ok. response: <Response [403]>.
    Downloaded the image. ==> downloads/orchid/232456.jpg (1).jpg
    Downloaded the image. ==> downloads/orchid/scale_1200 (14).jpg
    Downloaded the image. ==> downloads/orchid/344546-svetik.jpg (1).jpg
    fail: https://catherineasquithgallery.com/uploads/posts/2021-02/1612982236_77-p-fon-orkhideya-rozovaya-107.jpg error: img_url response is not ok. response: <Response [404]>.
    Downloaded the image. ==> downloads/orchid/2812807.jpg (1).jpg
    fail: https://mocah.org/uploads/posts/545724-beautiful-bloom.jpg error: img_url response is not ok. response: <Response [403]>.
    fail: https://catherineasquithgallery.com/uploads/posts/2021-02/1612982256_84-p-fon-orkhideya-rozovaya-117.

### Do some cleanup
Now I just want to move all files to the corresponding flowers directory and remove all duplicate files. Duplicates are downloaded with a space and number in the name (i.e., 'file (1).jpg' would be a duplicate of 'file.jpg').

In [12]:
! mv downloads/orchid/* flowers/orchid/.

In [13]:
! rm -rf downloads

In [14]:
! rm flowers/orchid/*\ *